In [1]:
# Itinializing the Jopyter Notebook file VacationPy.ipynb

In [2]:
# Importing relevant dependencies 
import requests
import json
import pandas as pd
import numpy as np
from config import api_key

# Importing gmaps and Google API key
import gmaps
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [3]:
# Read in cities from WeatherPy output
city_data=pd.read_csv("../WeatherPy/city_data.csv")
city_data.head()

,City name,Country name,Longitude,Latitude,"Temperature, F","Humidity, %","Cloudiness, %","Wind speed, mph",Hemisphere
0,aasiaat,gl,-53.464459,71.022578,35.53,85,96,2.75,Northern
1,aban,ru,96.046101,57.006436,44.65,50,4,2.71,Northern
2,abu samrah,qa,49.980118,20.223969,95.22,11,0,4.97,Northern
3,acapulco,mx,-107.577970,2.133108,82.83,55,100,0.94,Northern
4,acari,pe,-75.100356,-18.049583,74.16,59,95,2.32,Southern


In [4]:
# Set figure dimensions
figure_layout = {
    'width': '100%',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [5]:
# Create a list, which would contain lon, lat coordinates

In [6]:
# Set up gmap with a marker layer containing heatmap of the humidity (%)
locations = city_data[['Latitude', 'Longitude']]
weights = city_data['Humidity, %']
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=200,
                                 point_radius=4))
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [7]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

# Set up weather conditions to filter city_data dataset
max_temp=75.0
min_temp=70.0
max_wind=10.0
max_cloud=0.0

city_data_ideal=city_data[(city_data['Temperature, F']  >  min_temp)  & 
                          (city_data['Temperature, F']  <  max_temp)  &
                          (city_data['Cloudiness, %']   == max_cloud) &
                          (city_data['Wind speed, mph'] <  max_wind)]


In [8]:
len(city_data), len(city_data_ideal)

(1000, 12)

In [16]:
# Use lat,lon coordinates of select cities to place markers
ideal_locations = city_data_ideal[['Latitude', 'Longitude']]
print(city_data_ideal['City name'] + ", " + city_data_ideal['Country name'])

city_label_list=(city_data_ideal['City name'] + ", " + city_data_ideal['Country name']).tolist()

# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(ideal_locations,
    info_box_content=[f"Testing: {city}" for city in city_label_list])
fig.add_layer(markers)
fig

17         albacete, es
33        ambovombe, mg
91           beloha, mg
248       ewa beach, us
470         lubango, ao
723    richards bay, za
762        san juan, ar
839         susehri, tr
973         xixiang, cn
983        yinchuan, cn
986           yumen, cn
991         zhangye, cn
dtype: object


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [10]:
# find the closest hotel within 5000 meters of city coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords
    "rankby": "distance",
    "type": "hotel",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in types_df.iterrows():

    # get restaurant type from df
    restr_type = row['ethnicity']

    # add keyword to params dict
    params['keyword'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

NameError: name 'types_df' is not defined